# Connecting to a port in the container

Another common use case is that we start some kind of long-running process in the container, and talk to it through a port. That process could be a Jupyter Notebook, for example. 

## Example http.server

For demonstration purposes, we will use Python's in-built web server. To run it from the host, we could use

In [1]:
# NBVAL_SKIP
!python -m http.server

Serving HTTP on 0.0.0.0 port 8000 (http://0.0.0.0:8000/) ...
127.0.0.1 - - [29/Jun/2017 09:32:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2017 09:32:46] code 404, message File not found
127.0.0.1 - - [29/Jun/2017 09:32:46] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [29/Jun/2017 09:32:48] "GET /000-introduction.ipynb HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2017 09:32:52] "GET /cow-input.txt HTTP/1.1" 200 -
^C

Keyboard interrupt received, exiting.


This program will show the contents of the current file system in a webbrowser interface at port 8000 of this machine. So typically at one or some of these links http://127.0.0.1:8000 or http://localhost:8000 or http://0.0.0.0:8000).

(If you have executed the above cell by pressing SHIFT+RETURN, you need to interrupt the http.server process to get the control back in the notebook. This can be done by choosing from the menu "Kernel" -> "Interrupt".)

We will now create a container and run this server inside the container. We like to use a webbrowser on the host machine to inspect the files.

First, we create the Dockerfile:

In [2]:
%%file Dockerfile
FROM ubuntu:16.04

RUN apt-get -y update
RUN apt-get -y install python3

CMD python3 -m http.server

Overwriting Dockerfile


The last line starts the `http.server` when the container is run.

In [3]:
#NBVAL_IGNORE_OUTPUT
!docker build -t portdemo .

Sending build context to Docker daemon 122.9 kB
Step 1/4 : FROM ubuntu:16.04
 ---> 7b9b13f7b9c0
Step 2/4 : RUN apt-get -y update
 ---> Running in 5fb58fcdc349
Get:1 http://archive.ubuntu.com/ubuntu xenial InRelease [247 kB]
Get:2 http://archive.ubuntu.com/ubuntu xenial-updates InRelease [102 kB]
Get:3 http://archive.ubuntu.com/ubuntu xenial-backports InRelease [102 kB]
Get:4 http://security.ubuntu.com/ubuntu xenial-security InRelease [102 kB]
Get:5 http://archive.ubuntu.com/ubuntu xenial/universe Sources [9802 kB]
Get:6 http://security.ubuntu.com/ubuntu xenial-security/universe Sources [39.9 kB]
Get:7 http://security.ubuntu.com/ubuntu xenial-security/main amd64 Packages [367 kB]
Get:8 http://archive.ubuntu.com/ubuntu xenial/main amd64 Packages [1558 kB]
Get:9 http://archive.ubuntu.com/ubuntu xenial/restricted amd64 Packages [14.1 kB]
Get:10 http://archive.ubuntu.com/ubuntu xenial/universe amd64 Packages [9827 kB]
Get:11 http://security.ubuntu.com/ubuntu xenial-security/restricted amd64

Selecting previously unselected package file.
Preparing to unpack .../file_1%3a5.25-2ubuntu1_amd64.deb ...
Unpacking file (1:5.25-2ubuntu1) ...
Processing triggers for libc-bin (2.23-0ubuntu7) ...
Setting up mime-support (3.59ubuntu1) ...
Setting up libmpdec2:amd64 (2.4.2-1) ...
Setting up libsqlite3-0:amd64 (3.11.0-1ubuntu1) ...
Setting up libpython3.5-stdlib:amd64 (3.5.2-2ubuntu0~16.04.1) ...
Setting up python3.5 (3.5.2-2ubuntu0~16.04.1) ...
Setting up libpython3-stdlib:amd64 (3.5.1-3) ...
Setting up libmagic1:amd64 (1:5.25-2ubuntu1) ...
Setting up file (1:5.25-2ubuntu1) ...
Setting up python3 (3.5.1-3) ...
running python rtupdate hooks for python3.5...
running python post-rtupdate hooks for python3.5...
Setting up dh-python (2.20151103ubuntu1.1) ...
Processing triggers for libc-bin (2.23-0ubuntu7) ...
 ---> bcae71a4eeee
Removing intermediate container 04715eabb040
Step 4/4 : CMD python3 -m http.server
 ---> Running in e2e584b4cd02
 ---> c20c37734cb0
Removing intermediate container e

We now need to export the port 8000 in the container. We can do this using:

In [4]:
#NBVAL_SKIP
!docker run -p 8123:8000 portdemo

^C


The numbers following reflect the internal port (8000) that should be connected to the port (8123) on the host system.

Once the above command is executing, we should be able to browse the file system in the container by going to the link http://localhost:8123 (or http://127.0.0.1:8123 or http://0.0.0.0:8123) on the host system. 

We could have mapped port 8000 in the container to port 8000 on the host as well (`-p 8000:8000`).

As before, to top the process, select `Kernel->Interrupt`.

## Jupyter Notebook

A common application of exposing ports is to install computational or data analysis software inside the container, and to control it from a Jupyter notebook running inside the container, but to use a webbrowser from the host system to interact with the notebook. In that case, the above example of exposing the port is in principle the right way to go, too. However, as this is a common usecase, there are a number of prepared Dockerfiles to install the notebook inside the container available at https://github.com/jupyter/docker-stacks, so tat one can start the Dockerfile with `FROM jupyter/...`, (instead of `FROM ubuntu/...`) and in this way build on the Dockerfiles that the Jupyter team provides already.

The container image for https://github.com/jupyter/docker-stacks/tree/master/scipy-notebook might be a good starting point for work based on the Scientific Python stack.